In [11]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('src'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [129]:
import pickle
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np

In [5]:
LOAD_PATH = '../data/interim/'
LOAD_ANALYSIS_DF = '4.0-preprocessed-data-analysation.pkl'
LOAD_SKILLS_DEV = '7.0-Chosen_features_and_roles.pkl'

In [6]:
survey = pd.read_pickle(LOAD_PATH + LOAD_ANALYSIS_DF)
skills_dev_df = pd.read_pickle(LOAD_PATH + LOAD_SKILLS_DEV)

In [7]:
with open(LOAD_PATH + 'chosen_columns.pkl', 'rb') as f:
    chosen_columns = pickle.load(f)
survey = survey[chosen_columns['analysis']]


In [8]:
survey

,MainBranch,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,CompTotal,...,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSysProfessional use,VersionControlSystem,Age,Gender
2,0,[Employed_full-time],"Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",14,5,[Data scientist or machine learning specialist...,20 to 99 employees,United Kingdom of Great Britain and Northern I...,32000.0,...,[Pandas],[.NET],[],[],"[Notepad++, Visual Studio]","[Notepad++, Visual Studio]",[Windows],[Git],25-34 years old,[Man]
3,1,[Employed_full-time],Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",20,17,[Developer_full-stack],100 to 499 employees,Israel,60000.0,...,[.NET],[.NET],[],[],"[Notepad++, Visual Studio, Visual Studio Code]","[Notepad++, Visual Studio, Visual Studio Code]",[Windows],[Git],35-44 years old,[Man]
7,1,[Not employed_but looking for work],NaN,Some college/university study without earning ...,1,0,"[Developer_full-stack, Student]",NaN,India,NaN,...,[],[],[npm],"[Unity 3D, Yarn]","[Atom, CLion, Eclipse, IntelliJ, Notepad++, Vi...","[Android Studio, IPython/Jupyter, Sublime Text...","[Linux-based, macOS]",[Git],18-24 years old,[Man]
9,1,[Independent contractor_freelancer_or self-emp...,Fully remote,Some college/university study without earning ...,37,30,"[Developer_desktop or enterprise applications,...","Just me - I am a freelancer, sole proprietor, ...",Croatia,NaN,...,[],[],[],[],"[Android Studio, RAD Studio (Delphi_C++ Builde...","[Android Studio, RAD Studio (Delphi_C++ Builde...",[Windows],[Git],45-54 years old,[Woman]
10,1,[Employed_full-time],"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,2,"[Developer_full-stack, Developer_back-end]",100 to 499 employees,United Kingdom of Great Britain and Northern I...,48000.0,...,[.NET],[],"[Docker, npm, Terraform]","[Docker, Terraform]","[Rider, Visual Studio, Visual Studio Code]","[Rider, Visual Studio Code]","[Windows, Windows Subsystem for Linux (WSL)]",[Git],18-24 years old,[Man]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73262,0,"[Student_full-time, Employed_part-time]","Hybrid (some remote, some in-person)","Secondary school (e.g. American high school, G...",4,0,"[Developer_front-end, Student]",NaN,Germany,NaN,...,[],[],[npm],[],"[Eclipse, Visual Studio Code]","[Android Studio, Visual Studio Code]",[],[Git],18-24 years old,[Man]
73263,1,[Employed_full-time],Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",8,5,[Developer_back-end],100 to 499 employees,Nigeria,60000.0,...,[Flutter],[],"[Docker, Homebrew, Kubernetes, npm]","[Docker, Homebrew, Kubernetes, npm]","[IPython/Jupyter, Sublime Text, Vim, Visual St...","[Sublime Text, Vim, Visual Studio Code]",[macOS],[Git],25-34 years old,[Man]
73264,1,[Employed_full-time],Full in-person,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,5,[Data scientist or machine learning specialist],I don’t know,United States of America,107000.0,...,"[Keras, NumPy, Pandas, Scikit-learn, TensorFlo...","[NumPy, Pandas, Torch/PyTorch, Hugging Face Tr...",[],[],"[IPython/Jupyter, Notepad++, Spyder, Vim, Visu...","[Notepad++, Spyder, Vim, Visual Studio Code]","[Linux-based, Windows]",[Git],25-34 years old,[Man]
73265,0,[Employed_full-time],"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",42,33,"[Developer_full-stack, Developer_desktop or en...",20 to 99 employees,United States of America,NaN,...,"[.NET, Pandas, React Native]","[.NET, Cordova, Ionic, Pandas, React Native, X...",[npm],"[npm, Unreal Engine]","[Spyder, Visual Studio, Visual Studio Code]","[Spyder, Visual Studio, Visual Studio Code]",[Windows],[Git],55-64 years old,[Man]


In [191]:
survey.Country.value_counts().sort_values()

Niger                                                      1
Seychelles                                                 1
Solomon Islands                                            1
Palau                                                      1
Djibouti                                                   1
                                                        ... 
Canada                                                  1771
United Kingdom of Great Britain and Northern Ireland    3097
Germany                                                 3687
India                                                   4442
United States of America                                9826
Name: Country, Length: 177, dtype: int64

# 1.Employment vs Dev type
first we need to know Employment type abd is there any relation between them with dev type or not

In [112]:
def binarize(df, column):
    binarizer = MultiLabelBinarizer()
    mask = df[column].notnull()

    #filter by boolean indexing
    arr = binarizer.fit_transform(df.loc[mask, column])

    #create DataFrame and add missing (NaN)s index values
    return  (pd.DataFrame(arr, index=df.index[mask], columns=binarizer.classes_)
           .reindex(df[column].index, fill_value=0))


In [176]:
def change_labels(x: str):
    if 'Developer_' in x:
        x = x.replace('Developer_', '') + ' dev'

    if ' or ' in x:
        index_to_delete = x.index(' or ')
        x = x[index_to_delete + 4:]

    if 'Engineer_' in x:
        x = x.replace('Engineer_', '') + '  Engineer'

    if x.__contains__('back_end') or x.__contains__('full_stack'):
        x += ' dev'
    return x

In [117]:
employment_df = binarize(survey, 'Employment')
employment_df

,Employed_full-time,Employed_part-time,Independent contractor_freelancer_or self-employed,Not employed_and not looking for work,Not employed_but looking for work,Retired,Student_full-time,Student_part-time
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0
9,0,0,1,0,0,0,0,0
10,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
73262,0,1,0,0,0,0,1,0
73263,1,0,0,0,0,0,0,0
73264,1,0,0,0,0,0,0,0
73265,1,0,0,0,0,0,0,0


In [269]:
def Employment_vs_Dev_type(dev_types):
    employment_chosen_columns = ['Employed_full-time', 'Employed_part-time',
                             'Independent contractor_freelancer_or self-employed']

    # df = survey[survey.Country.isin(countries)]
    employment_df = binarize(survey, 'Employment')
    employment_df = employment_df[employment_chosen_columns]
    employment_df.rename(columns = {'Independent contractor_freelancer_or self-employed':'freelancer',
                                    'Employed_full-time' : 'full-time',
                                    'Employed_part-time': 'part-time'}, inplace = True)

    employment_df.columns = pd.MultiIndex.from_product([['Employment'], employment_df.columns])
    dev_df = skills_dev_df['DevType'][dev_types]
    dev_df.columns = pd.MultiIndex.from_product([['DevType'], dev_df.columns])
    employment_dev = employment_df.merge(dev_df, left_index=True, right_index=True)
    employment_dev_count = {}
    for employment_type in employment_dev.Employment.columns:
        employment_dev_count[employment_type] = ((employment_dev.Employment[employment_type].values.reshape(1,-1)) @ employment_dev.DevType )
    employment_dev_count = pd.concat(employment_dev_count)
    employment_dev_percentage = (employment_dev_count / employment_dev_count.sum(axis=0)).applymap('{:,.2f}'.format).astype('float')


    layout = go.Layout(
                title="Employment_vs_job_type",
                yaxis=dict(
                    title="job_type"),
                xaxis=dict(
                    title="percentage %"),
                barmode='group',
                plot_bgcolor='rgb(233,233,233)')
    fig = go.Figure(data=[
        go.Bar(name= i[0], y=np.arange(0,len(dev_types)), x=employment_dev_percentage.loc[i , :].values,orientation='h') for i in employment_dev_percentage.index],

                    layout=layout)
    fig.update_layout(barmode='group',title_x = .5)
    # fig.update_xaxes(type="log")
    fig.update_layout(yaxis = dict(
            tickmode = 'array',
            tickvals = np.arange(0,len(dev_types)).tolist(),
            ticktext = [change_labels(x) for x in dev_types],
            tickfont = {'size' : 20},
            title_font_size = 25
        ),title_font_size = 30
        ,xaxis = dict(tickfont = {'size' : 20},title_font_size = 25 ))

    fig.update_layout(legend = dict(font = dict(family = "Courier", size = 15, color = "black")),
              legend_title = dict(font = dict(family = "Courier", size = 30, color = "blue")))

    # fig.update_layout({
    #     'plot_bgcolor':'rgba(0, 0, 0, 0)',
    #     'paper_bgcolor':'rgba(0, 0, 0, 0)',
    # })

    fig.update_traces(textfont_size = 20)

    return fig

In [270]:
dev_types = ['Developer_full-stack', 'Developer_back-end', 'Developer_front-end', 'Data scientist or machine learning specialist', 'Engineer_data', 'Cloud infrastructure engineer']

In [271]:
Employment_vs_Dev_type(dev_types)

In [258]:
survey.Country.isin(['Italy', 'Canada'])

2        False
3        False
7        False
9        False
10       False
         ...  
73262    False
73263    False
73264    False
73265    False
73266    False
Name: Country, Length: 50423, dtype: bool

In [125]:
skills_dev_df.DevType.isin(['Academic researcher', 'Developer_front-end', 'Engineer_data'])

,Academic researcher,Blockchain,Cloud infrastructure engineer,Data or business analyst,Data scientist or machine learning specialist,DevOps specialist,Developer_back-end,Developer_desktop or enterprise applications,Developer_embedded applications or devices,Developer_front-end,...,full_stack_Java,full_stack_.JavaScript,full_stack_PHP,full_stack_python,back_end_Java,back_end_.JavaScript,back_end_.NET,back_end_C++,back_end_python,back_end_PHP
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73262,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73263,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73264,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73265,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
